## 3 Pandas进阶

In [1]:
import pandas as pd
import numpy as np

In [ ]:
#一个参数 默认起点0，步长为1 输出：[0 1 2]
a = np.arange(3)

#两个参数 默认步长为1 输出[3 4 5 6 7 8]
a = np.arange(3,9)

#三个参数 起点为0，终点为3，步长为0.1 输出[ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1  1.2  1.3  1.4 1.5  1.6  1.7  1.8  1.9  2.   2.1  2.2  2.3  2.4  2.5  2.6  2.7  2.8  2.9]
a = np.arange(0, 3, 0.1)

### 3.1 数据重塑和轴向旋转

#### （1）层次化索引

层次化索引是pandas的一项重要功能，他能使我们再一个轴上拥有多个索引

**Series的层次化索引**<br/>
两层索引

In [4]:
s=pd.Series(np.arange(1,10),index=[['a','a','a','b','b','c','c','d','d'],[1,2,3,1,2,3,1,2,3]])
s
#有点类似合并单元格？

a  1    1
   2    2
   3    3
b  1    4
   2    5
c  3    6
   1    7
d  2    8
   3    9
dtype: int32

In [5]:
s.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('c', 3),
            ('c', 1),
            ('d', 2),
            ('d', 3)],
           )

In [6]:
s['a']
# 取外层索引

1    1
2    2
3    3
dtype: int32

In [7]:
s['a':'c']
# 切片 a到c

a  1    1
   2    2
   3    3
b  1    4
   2    5
c  3    6
   1    7
dtype: int32

In [12]:
s[:,1]
# 取内存索引

a    1
b    4
c    7
dtype: int32

通过unstack方法可以将Series变成一个DataFrame <br/>
unstack不堆叠  转换成表格

In [13]:
s.unstack()

,1,2,3
a,1.0,2.0,3.0
b,4.0,5.0,NaN
c,7.0,NaN,6.0
d,NaN,8.0,9.0


In [8]:
s.unstack().stack()

a  1    1.0
   2    2.0
   3    3.0
b  1    4.0
   2    5.0
c  1    7.0
   3    6.0
d  2    8.0
   3    9.0
dtype: float64

#### （2）DataFrame的层次化索引

对于DataFrame来说，行和列都能够进行层次化索引。

In [9]:
data=pd.DataFrame(np.arange(12).reshape(4,3))
data

,0,1,2
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11


In [10]:
data=pd.DataFrame(np.arange(12).reshape(4,3),index=[['a','a','b','b'],[1,2,1,2]])
data

0   1   2
a 1  0   1   2
  2  3   4   5
b 1  6   7   8
  2  9  10  11

In [11]:
data=pd.DataFrame(np.arange(12).reshape(4,3),index=[['a','a','b','b'],[1,2,1,2]],columns=[['A','A','B'],['Z','X','C']])
data
# 有四个索引 决定一个值 二维数据变成了四维数据

A       B
     Z   X   C
a 1  0   1   2
  2  3   4   5
b 1  6   7   8
  2  9  10  11

选取列

In [13]:
data['A']

Z   X
a 1  0   1
  2  3   4
b 1  6   7
  2  9  10

设置索引名称

In [14]:
data.index.names=['row1','row2']
data

A       B
           Z   X   C
row1 row2           
a    1     0   1   2
     2     3   4   5
b    1     6   7   8
     2     9  10  11

设置列级别名称

In [15]:
data.columns.names=['column1','column2']
data

column1    A       B
column2    Z   X   C
row1 row2           
a    1     0   1   2
     2     3   4   5
b    1     6   7   8
     2     9  10  11

行顺序调整 row1，row2交换顺序

In [25]:
data.swaplevel('row1','row2')

column1    A       B
column2    Z   X   C
row2 row1           
1    a     0   1   2
2    a     3   4   5
1    b     6   7   8
2    b     9  10  11

##### 层次化——电影数据示列

数据导入

In [31]:
df=pd.read_excel('movie_data2.xlsx')
df.head()

,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,产地,上映时间,时长,年代,评分,首映地点
0,0,0.0,肖申克的救赎,692795,剧情/犯罪,美国,1994-09-10 00:00:00,142,1994,9.6,多伦多电影节
1,1,1.0,控方证人,42995,剧情/悬疑/犯罪,美国,1957-12-17 00:00:00,116,1957,9.5,美国
2,2,2.0,美丽人生,327855,剧情/喜剧/爱情,意大利,1997-12-20 00:00:00,116,1997,9.5,意大利
3,3,3.0,阿甘正传,580897,剧情/爱情,美国,1994-06-23 00:00:00,142,1994,9.4,洛杉矶首映
4,4,4.0,霸王别姬,478523,剧情/爱情/同性,中国大陆,1993-01-01 00:00:00,171,1993,9.4,香港


In [27]:
df.index

RangeIndex(start=0, stop=38724, step=1)

<font color=black size=4 face=雅黑>set_index：可以把列变成索引<br>
reset_index：把索引变成列</font>

把产地和年代同时设置成索引，产地是外层索引，年代是内层索引

In [32]:
df=df.set_index(['产地','年代'])
df.head()

Unnamed: 0  Unnamed: 0.1      名字    投票人数        类型  \
产地   年代                                                         
美国   1994           0           0.0  肖申克的救赎  692795     剧情/犯罪   
     1957           1           1.0    控方证人   42995  剧情/悬疑/犯罪   
意大利  1997           2           2.0   美丽人生   327855  剧情/喜剧/爱情   
美国   1994           3           3.0    阿甘正传  580897     剧情/爱情   
中国大陆 1993           4           4.0    霸王别姬  478523  剧情/爱情/同性   

                          上映时间   时长   评分    首映地点  
产地   年代                                           
美国   1994  1994-09-10 00:00:00  142  9.6  多伦多电影节  
     1957  1957-12-17 00:00:00  116  9.5      美国  
意大利  1997  1997-12-20 00:00:00  116  9.5     意大利  
美国   1994  1994-06-23 00:00:00  142  9.4   洛杉矶首映  
中国大陆 1993  1993-01-01 00:00:00  171  9.4      香港

<font color=black size=4 face=雅黑>每一个索引都是一个元组</font>

In [35]:
df.index[0]

('美国', 1994)

获取所有的美国电影，由于产地信息已经变成了索引，因此可以用loc方法

In [36]:
df.loc['美国'].head()

,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点
年代,,,,,,,,,
1994,0,0.0,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节
1957,1,1.0,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国
1994,3,3.0,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映
2012,5,5.0,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.4,中国大陆
1993,6,6.0,辛德勒的名单,306904,剧情/历史/战争,1993-11-30 00:00:00,195,9.4,华盛顿首映


In [ ]:
此时的索引已经变成了年代

**这样做最大的好处就是我们可以简化很多的筛选环节**

**每一个索引是一个元组**

<font color=red size=4>索引交换 swaplevel </font>

In [38]:
df=df.swaplevel('产地','年代')
df.head()

,,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点
年代,产地,,,,,,,,,
1994,美国,0,0.0,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节
1957,美国,1,1.0,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国
1997,意大利,2,2.0,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利
1994,美国,3,3.0,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映
1993,中国大陆,4,4.0,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港


In [40]:
df.loc[1994]
#此时产地成了索引

,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点
产地,,,,,,,,,
美国,0,0.0,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节
美国,3,3.0,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映
法国,9,9.0,这个杀手不太冷,662552,剧情/动作/犯罪,1994-09-14 00:00:00,133,9.4,法国
美国,314,314.0,34街的,768,剧情/家庭/奇幻,1994-12-23 00:00:00,114,7.9,美国
中国大陆,402,402.0,活着,202794,剧情/家庭,1994-05-18 00:00:00,132,9.0,法国
...,...,...,...,...,...,...,...,...,...
美国,38203,38217.0,鬼精灵2： 恐怖,60,喜剧/恐怖/奇幻,1994-04-08 00:00:00,85,5.8,美国
英国,38333,38347.0,黑色第16,44,剧情/惊悚,1996-02-01 00:00:00,106,6.8,美国
日本,38677,38691.0,蜡笔小新之布里布里王国的秘密宝藏 クレヨンしんちゃん ブリブリ王国の,2142,动画,1994-04-23 00:00:00,94,7.7,日本


取消层次化索引 也就是将索引变成了列

In [41]:
df=df.reset_index()

In [42]:
df.head()

,年代,产地,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点
0,1994,美国,0,0.0,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节
1,1957,美国,1,1.0,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国
2,1997,意大利,2,2.0,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利
3,1994,美国,3,3.0,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映
4,1993,中国大陆,4,4.0,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港


#### （3）数据旋转

行列转化

In [44]:
data=df[:5]#先取前五行
data

,年代,产地,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点
0,1994,美国,0,0.0,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节
1,1957,美国,1,1.0,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国
2,1997,意大利,2,2.0,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利
3,1994,美国,3,3.0,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映
4,1993,中国大陆,4,4.0,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港


T 可以直接让数据的行和列进行交换

In [45]:
data.T

,0,1,2,3,4
年代,1994,1957,1997,1994,1993
产地,美国,美国,意大利,美国,中国大陆
Unnamed: 0,0,1,2,3,4
Unnamed: 0.1,0,1,2,3,4
名字,肖申克的救赎,控方证人,美丽人生,阿甘正传,霸王别姬
投票人数,692795,42995,327855,580897,478523
类型,剧情/犯罪,剧情/悬疑/犯罪,剧情/喜剧/爱情,剧情/爱情,剧情/爱情/同性
上映时间,1994-09-10 00:00:00,1957-12-17 00:00:00,1997-12-20 00:00:00,1994-06-23 00:00:00,1993-01-01 00:00:00
时长,142,116,116,142,171
评分,9.6,9.5,9.5,9.4,9.4


dataframe也可以使用stack和unstack，转化为层次索引的Series

In [46]:
data.stack()

0  年代                             1994
   产地                               美国
   Unnamed: 0                        0
   Unnamed: 0.1                      0
   名字                           肖申克的救赎
   投票人数                         692795
   类型                            剧情/犯罪
   上映时间            1994-09-10 00:00:00
   时长                              142
   评分                              9.6
   首映地点                         多伦多电影节
1  年代                             1957
   产地                               美国
   Unnamed: 0                        1
   Unnamed: 0.1                      1
   名字                             控方证人
   投票人数                          42995
   类型                         剧情/悬疑/犯罪
   上映时间            1957-12-17 00:00:00
   时长                              116
   评分                              9.5
   首映地点                             美国
2  年代                             1997
   产地                              意大利
   Unnamed: 0                        2
   Unnamed: 0.1          

In [47]:
data.stack().unstack()

,年代,产地,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点
0,1994,美国,0,0,肖申克的救赎,692795,剧情/犯罪,1994-09-10,142,9.6,多伦多电影节
1,1957,美国,1,1,控方证人,42995,剧情/悬疑/犯罪,1957-12-17,116,9.5,美国
2,1997,意大利,2,2,美丽人生,327855,剧情/喜剧/爱情,1997-12-20,116,9.5,意大利
3,1994,美国,3,3,阿甘正传,580897,剧情/爱情,1994-06-23,142,9.4,洛杉矶首映
4,1993,中国大陆,4,4,霸王别姬,478523,剧情/爱情/同性,1993-01-01,171,9.4,香港


### 3.2 数据分组、分组运算

<font color=blue size=4 face=雅黑>GroupBy技术：实现数据的分组，和分组运算，作用类似于数据透视表</font>

按照电影产地进行分组

**先定义一个分组变量group**

In [48]:
group=df.groupby(df['产地'])

In [49]:
group

**可以计算分组后的各个统计量**

In [50]:
group.mean().head()

,年代,Unnamed: 0,Unnamed: 0.1,投票人数,时长,评分
产地,,,,,,
中国台湾,1999.009709,19347.205502,19352.632686,8474.864078,87.257282,7.066667
中国大陆,2004.599684,20388.198843,20393.987638,10898.293793,81.432930,6.064703
中国香港,1991.100316,19226.836198,19232.110488,8167.446159,88.541214,6.474114
丹麦,1999.090909,20412.055556,20417.853535,1993.858586,88.101010,7.245960
俄罗斯,1984.899371,20081.157233,20086.800839,1019.134172,95.918239,7.554507


计算每年的平均评分

In [51]:
df['评分'].groupby(df['年代']).mean().head()

年代
1888    7.950000
1890    4.800000
1892    7.500000
1894    6.633333
1895    7.575000
Name: 评分, dtype: float64

**只会对数值变量进行分组运算**

In [53]:
#对年代求平均数、求和并没有什么意义，这里将年代转成string类型
df['年代']=df['年代'].astype('str')

In [54]:
df.groupby(df['产地']).mean().head()
# 这里对年代就没有均值的求取了

,Unnamed: 0,Unnamed: 0.1,投票人数,时长,评分
产地,,,,,
中国台湾,19347.205502,19352.632686,8474.864078,87.257282,7.066667
中国大陆,20388.198843,20393.987638,10898.293793,81.432930,6.064703
中国香港,19226.836198,19232.110488,8167.446159,88.541214,6.474114
丹麦,20412.055556,20417.853535,1993.858586,88.101010,7.245960
俄罗斯,20081.157233,20086.800839,1019.134172,95.918239,7.554507


我们也可以传入多个分组变量

In [56]:
df.groupby([df['产地'],df['年代']]).mean()

Unnamed: 0  Unnamed: 0.1          投票人数          时长        评分
产地   年代                                                                  
中国台湾 1963  14805.000000  14806.000000    121.000000  113.000000  6.400000
     1965  30276.000000  30285.000000    153.666667  105.000000  6.800000
     1966  26299.000000  26308.000000     51.000000   60.000000  7.900000
     1967  16605.000000  16606.000000   4444.000000  112.000000  8.000000
     1968  21276.500000  21285.500000     89.000000   83.000000  7.400000
...                 ...           ...           ...         ...       ...
韩国   2012  17576.207547  17580.452830   5762.537736  100.669811  6.064151
     2013  18953.117117  18958.252252  10189.036036   96.504505  6.098198
     2014  19881.700000  19887.200000   3776.266667   98.666667  5.650833
     2015  19036.128440  19040.908257   3209.247706  100.266055  5.423853
     2016  18901.350000  18906.100000   1739.850000  106.100000  5.730000

[1585 rows x 5 columns]

**获得每个地区，每一年的电影的评分的均值**

In [77]:
df['评分'].groupby([df['产地'],df['年代']]).mean()

产地    年代  
中国台湾  1963    6.400000
      1965    6.800000
      1966    7.900000
      1967    8.000000
      1968    7.400000
                ...   
韩国    2012    6.064151
      2013    6.098198
      2014    5.650833
      2015    5.423853
      2016    5.730000
Name: 评分, Length: 1511, dtype: float64

<font color='red' size=3>Series通过unstack方法转化为dataframe——会产生缺失值</font>

In [57]:
df['评分'].groupby([df['产地'],df['年代']]).mean().unstack().head()

年代,1888,1890,1892,1894,1895,1896,1897,1898,1899,1900,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
产地,,,,,,,,,,,,,,,,,,,,,
中国台湾,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.420000,7.100000,7.053846,7.231818,6.556098,7.076471,6.522222,6.576000,NaN,NaN
中国大陆,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.589726,6.536628,6.357831,5.824606,5.727187,5.316667,4.963757,4.969189,4.712000,NaN
中国香港,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.476786,6.442553,6.245455,5.971053,6.110526,6.105714,5.616667,5.589189,5.390909,NaN
丹麦,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.050000,7.118182,7.362500,7.016667,7.418750,6.555556,7.120000,7.166667,7.000000,NaN
俄罗斯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.741667,6.278571,6.858824,7.040000,6.572727,6.875000,7.175000,7.342857,NaN,NaN


### 3.3 离散化处理

再实际的数据分析项目中，对有的数据属性，我们往往并不关注数据的绝对取值，只关注它所处的区间或等级。<br>
比如我们可以把评分9分及以上定义为A，7-9分为B，5-7分为C，3-5分为D，小于3分的为E。

<font color=blue size=3>离散化也可以称为分组、区间化

Pandas为我们提供了方便的函数cut()：<br>
**pd.cut (x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False)<br>**
参数：<br>
x：需要离散化的数组、Series、DataFrame对象<br>
bins：分组的依据<br>
right=True：默认包括右端点<br>
include_lowest=False：默认不包括左端点<br>
labels：是否要用标记替换分组<br>
retbins：返回x中对应的bins的列表<br>
precision：精度

In [58]:
df['评分等级']=pd.cut(df['评分'],[0,3,5,7,9,10],labels=['E','D','C','B','A'])
df

,年代,产地,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级
0,1994,美国,0,0.0,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.600000,多伦多电影节,A
1,1957,美国,1,1.0,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.500000,美国,A
2,1997,意大利,2,2.0,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.500000,意大利,A
3,1994,美国,3,3.0,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.400000,洛杉矶首映,A
4,1993,中国大陆,4,4.0,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.400000,香港,A
...,...,...,...,...,...,...,...,...,...,...,...,...
38719,1935,美国,38719,38734.0,1935年,57,喜剧/歌舞,1935-03-15 00:00:00,98,7.600000,美国,B
38720,1986,中国大陆,38720,38735.0,血溅画屏,95,剧情/悬疑/犯罪/武侠/古装,1905-06-08 00:00:00,91,7.100000,美国,B
38721,1986,中国大陆,38721,38736.0,魔窟中的幻想,51,惊悚/恐怖/儿童,1905-06-08 00:00:00,78,8.000000,美国,B
38722,1977,俄罗斯,38722,38737.0,列宁格勒围困之星火战役 Блокада: Фильм 2: Ленинградский ме...,32,剧情/战争,1905-05-30 00:00:00,97,6.600000,美国,C


**根据投票人数来刻画电影的热门<br>**
投票人数越多，热门程度越高

<font color=red >使用  np.percentile 进行分位数计算</font>
a = range(1,101)<br/>
#求取a数列第90%分位的数值<br/>
np.percentile(a, 90)<br/>
90.10000000000001<br/>

In [63]:
bins=np.percentile(df['投票人数'],[0,20,40,60,80,100])

In [64]:
df['热门程度']=pd.cut(df['投票人数'],bins,labels=['E','D','C','B','A'])

In [65]:
df

,年代,产地,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
0,1994,美国,0,0.0,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.600000,多伦多电影节,A,A
1,1957,美国,1,1.0,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.500000,美国,A,A
2,1997,意大利,2,2.0,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.500000,意大利,A,A
3,1994,美国,3,3.0,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.400000,洛杉矶首映,A,A
4,1993,中国大陆,4,4.0,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.400000,香港,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38719,1935,美国,38719,38734.0,1935年,57,喜剧/歌舞,1935-03-15 00:00:00,98,7.600000,美国,B,E
38720,1986,中国大陆,38720,38735.0,血溅画屏,95,剧情/悬疑/犯罪/武侠/古装,1905-06-08 00:00:00,91,7.100000,美国,B,D
38721,1986,中国大陆,38721,38736.0,魔窟中的幻想,51,惊悚/恐怖/儿童,1905-06-08 00:00:00,78,8.000000,美国,B,E
38722,1977,俄罗斯,38722,38737.0,列宁格勒围困之星火战役 Блокада: Фильм 2: Ленинградский ме...,32,剧情/战争,1905-05-30 00:00:00,97,6.600000,美国,C,E


大烂片：投票人数多，评分低

In [66]:
df[(df.热门程度=='A')&(df.评分等级=='E')]

,年代,产地,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
655,2011,中国大陆,655,655.0,B区,5187,剧情/惊悚/恐怖,2011-06-03 00:00:00,89,2.3,中国大陆,E,A
4376,2014,中国大陆,4376,4376.0,怖偶,4867,悬疑/惊悚,2014-05-07 00:00:00,88,2.8,中国大陆,E,A
5413,2011,中国大陆,5413,5413.0,床下有人,4309,悬疑/惊悚,2011-10-14 00:00:00,100,2.8,中国大陆,E,A
6802,2013,中国大陆,6802,6802.0,帝国秘符,4351,动作/冒险,2013-09-18 00:00:00,93,3.0,中国大陆,E,A
8232,2011,中国大陆,8232,8232.0,飞天,4764,剧情,2011-07-01 00:00:00,115,2.9,中国大陆,E,A
8406,2014,中国大陆,8406,8406.0,分手达人,3937,喜剧/爱情,2014-06-06 00:00:00,90,2.7,中国大陆,E,A
9601,2012,中国大陆,9601,9601.0,孤岛惊魂,2982,悬疑/惊悚/恐怖,2013-01-26 00:00:00,93,2.8,中国大陆,E,A
10513,2013,中国大陆,10513,10513.0,海天盛宴·韦口,3788,情色,2013-10-12 00:00:00,88,2.9,网络,E,A
16799,2013,中国大陆,16799,16800.0,孪生密码,6390,动作/悬疑,2013-11-08 00:00:00,96,2.9,中国大陆,E,A
21530,2010,日本,21530,21539.0,拳皇,6329,动作/科幻/冒险,2012-10-12 00:00:00,93,3.0,中国大陆,E,A


**冷门高分电影**

In [67]:
df[(df.热门程度=='E')&(df.评分等级=='A')]

,年代,产地,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
456,2015,美国,456,456.0,未知电影,45,音乐,2015-12-14 00:00:00,60,9.3,美国,A,E
587,2011,英国,587,587.0,BBC喜剧音,38,喜剧/音乐/歌舞,2011-08-13 00:00:00,95,9.3,美国,A,E
642,2003,美国,642,642.0,未知电影,38,纪录片/音乐,2003-02-03 00:00:00,55,9.2,美国,A,E
686,2014,英国,686,686.0,未知电影,63,音乐/舞台艺术,2014-05-18 00:00:00,49,9.5,美国,A,E
698,2008,英国,698,698.0,未知电影,52,纪录片,2008-07-25 00:00:00,40,9.8,美国,A,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37574,1986,英国,37574,37587.0,歌唱神探,36,剧情/悬疑/歌舞,1986-11-16 00:00:00,415,9.1,美国,A,E
37700,1972,美国,37700,37713.0,未知电影,56,纪录片,1972-12-23 00:00:00,70,9.3,美国,A,E
38105,1975,美国,38105,38119.0,山那边,70,剧情,1975-11-14 00:00:00,103,9.1,美国,A,E
38438,2015,美国,38438,38452.0,奎,62,纪录片/短片,2015-08-19 00:00:00,9,9.1,纽约电影论坛,A,E


将处理的数据保存

In [68]:
df.to_excel('movies_data3.xlsx')

### 3.4 合并数据集

#### （1）append

先把数据集拆分为多个，再进行合并

In [69]:
df_usa=df[df.产地=='美国']
df_cn=df[df.产地=='中国大陆']

In [70]:
df_cn.append(df_usa)

,年代,产地,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
4,1993,中国大陆,4,4.0,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.400000,香港,A,A
21,1961,中国大陆,21,21.0,大闹天宫,74881,动画/奇幻,1905-05-14 00:00:00,114,9.200000,上集,A,A
29,2015,中国大陆,29,29.0,穹顶之下,51113,纪录片,2015-02-28 00:00:00,104,9.200000,中国大陆,A,A
38,1982,中国大陆,38,38.0,茶馆,10678,剧情/历史,1905-06-04 00:00:00,118,9.200000,美国,A,A
45,1988,中国大陆,45,45.0,山水情,10781,动画/短片,1905-06-10 00:00:00,19,9.200000,美国,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38712,1987,美国,38712,38726.0,零下的激情,199,剧情/爱情/犯罪,1987-11-06 00:00:00,98,7.400000,美国,B,D
38714,1986,美国,38714,38728.0,离别秋波,240,剧情/爱情/音乐,1986-02-19 00:00:00,90,8.200000,美国,B,C
38717,1986,美国,38717,38732.0,极乐森林,45,纪录片,1986-09-14 00:00:00,90,8.100000,美国,B,E
38719,1935,美国,38719,38734.0,1935年,57,喜剧/歌舞,1935-03-15 00:00:00,98,7.600000,美国,B,E


#### （2）merge

pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
       left_index=False, right_index=False, sort=True,
       suffixes=('_x','_y'), copy=True, indicator=False)

left：对象<br>
right：另一个对象<br>
on：要加入的列。必须在左、右综合对象中找到。如果不能通过，left_index和right_index是假，将推断DataFrame中的列的交叉点为连接键<br>
left_on：从左边的综合使用作为键列。可以是列名或数组的长度等于长度综合。<br>
right_on：从右边的综合使用作为键列。可以是列名或数组的长度等于长度综合。<br>
left_index：如果为True，则使用索引（行标签）从左综合作为其连接键。在与多重（层次）的综合，级别数必须匹配连接键从右综合的数目。<br>
right_index：先沟通用法作为正确综合left_index。<br>
how：之一‘左’，‘右’，‘外在’，‘内部’。默认为内联。<br>
sort：综合通过连接键按字典顺序对结果进行排序。默认为True，设置为False在大多数情况下将提高性能。<br>
suffixes：字符串后缀并不适用于重叠列的元组。默认为（'_x'，'_y'）<br>
copy：即使重新索引是不必要总是从传递的综合对象，赋值的数据（默认为True）。在许多情况下不能避免，但可能会提高性能/内存使用情况，可以避免赋值上述案件有些病理，但尽管如此提供此选项。<br>
indicator：将列添加到输出综合呼吁_merge与信息源的每一行。_merge是绝对类型，并对观测其合并键只出现在‘左’的综合，关策其合并键只会出现在‘正确’的综合，和两个如果观测合并关键发现在两个 right_only  left_only 的值。

选取6部热门电影进行操作

In [72]:
df1=df.loc[:5]
df1

,年代,产地,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
0,1994,美国,0,0.0,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节,A,A
1,1957,美国,1,1.0,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国,A,A
2,1997,意大利,2,2.0,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利,A,A
3,1994,美国,3,3.0,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映,A,A
4,1993,中国大陆,4,4.0,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港,A,A
5,2012,美国,5,5.0,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.4,中国大陆,A,A


In [73]:
df2=df.loc[:5][['名字','产地']]#只想要名字和产地两列
df2['票房']=[111,222,333,444,555,666]
df2

,名字,产地,票房
0,肖申克的救赎,美国,111
1,控方证人,美国,222
2,美丽人生,意大利,333
3,阿甘正传,美国,444
4,霸王别姬,中国大陆,555
5,泰坦尼克号,美国,666


In [74]:
##打乱样本
df2=df2.sample(frac=1)
df2

,名字,产地,票房
5,泰坦尼克号,美国,666
2,美丽人生,意大利,333
4,霸王别姬,中国大陆,555
0,肖申克的救赎,美国,111
1,控方证人,美国,222
3,阿甘正传,美国,444


In [75]:
## 重新设置索引
df2.index=range(len(df2))
df2

,名字,产地,票房
0,泰坦尼克号,美国,666
1,美丽人生,意大利,333
2,霸王别姬,中国大陆,555
3,肖申克的救赎,美国,111
4,控方证人,美国,222
5,阿甘正传,美国,444


对df1和df2进行合并

In [76]:
pd.merge(df1,df2,how='inner',on='名字')

,年代,产地_x,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度,产地_y,票房
0,1994,美国,0,0.0,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节,A,A,美国,111
1,1957,美国,1,1.0,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国,A,A,美国,222
2,1997,意大利,2,2.0,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利,A,A,意大利,333
3,1994,美国,3,3.0,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映,A,A,美国,444
4,1993,中国大陆,4,4.0,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港,A,A,中国大陆,555
5,2012,美国,5,5.0,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.4,中国大陆,A,A,美国,666


两个样本都有 产地 列，所以会有两个产地列

##### （3）concat

将多个数据集进行批量合并

In [77]:
df1=df[:5]
df2=df[100:105]
df3=df[200:205]

In [78]:
df1

,年代,产地,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
0,1994,美国,0,0.0,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节,A,A
1,1957,美国,1,1.0,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国,A,A
2,1997,意大利,2,2.0,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利,A,A
3,1994,美国,3,3.0,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映,A,A
4,1993,中国大陆,4,4.0,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港,A,A


In [79]:
df2

,年代,产地,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
100,1993,韩国,100,100.0,101,146,喜剧/爱情,1993-06-19 00:00:00,112,7.4,韩国,B,D
101,1995,英国,101,101.0,10,186,喜剧,1995-01-25 00:00:00,101,7.4,美国,B,D
102,2013,韩国,102,102.0,素媛,114819,剧情/家庭,2013-10-02 00:00:00,123,9.1,韩国,A,A
103,2003,美国,103,103.0,101忠狗续集：伦敦,924,喜剧/动画/家庭,2003-01-21 00:00:00,70,7.5,美国,B,B
104,2000,美国,104,104.0,10,9514,喜剧/家庭,2000-09-22 00:00:00,100,7.0,美国,C,A


In [80]:
df3

,年代,产地,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
200,2014,日本,200,200.0,最完美的离婚 2014特别篇,18478,剧情/喜剧/爱情,2014-02-08 00:00:00,120,9.1,日本,A,A
201,2009,日本,201,201.0,2001夜物,84,剧情/动画,2009-10-02 00:00:00,80,6.6,美国,C,D
202,2009,中国香港,202,202.0,头七 頭,7039,恐怖,2009-05-21 00:00:00,60,6.2,美国,C,A
203,1896,法国,203,203.0,火车进站 L,7001,纪录片/短片,1896-01-06,60,8.8,法国,B,A
204,2009,美国,204,204.0,银行舞蹈,6944,短片,1905-07-01 00:00:00,60,7.8,美国,B,A


In [81]:
dff=pd.concat([df1,df2,df3],axis=0) ##默认axis=0 增加行数，axis=1增加列数
dff

,年代,产地,Unnamed: 0,Unnamed: 0.1,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
0,1994,美国,0,0.0,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节,A,A
1,1957,美国,1,1.0,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国,A,A
2,1997,意大利,2,2.0,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利,A,A
3,1994,美国,3,3.0,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映,A,A
4,1993,中国大陆,4,4.0,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港,A,A
100,1993,韩国,100,100.0,101,146,喜剧/爱情,1993-06-19 00:00:00,112,7.4,韩国,B,D
101,1995,英国,101,101.0,10,186,喜剧,1995-01-25 00:00:00,101,7.4,美国,B,D
102,2013,韩国,102,102.0,素媛,114819,剧情/家庭,2013-10-02 00:00:00,123,9.1,韩国,A,A
103,2003,美国,103,103.0,101忠狗续集：伦敦,924,喜剧/动画/家庭,2003-01-21 00:00:00,70,7.5,美国,B,B
104,2000,美国,104,104.0,10,9514,喜剧/家庭,2000-09-22 00:00:00,100,7.0,美国,C,A
